In [1]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
from pathlib import Path
from tbparse import SummaryReader


In [8]:
MODEL_TYPE='new_multitask_informer' 

SAVE_PATH = f"../results/{MODEL_TYPE}_results.md"
SEED=21
OVERWRITE=True
if OVERWRITE:
     open(SAVE_PATH, 'w').close()


In [6]:
def get_metric_results_from_eventfile(dirpath, metrics, SAVE_PATH="", title="", select="latest"):
    # select = "latest", or "lowest" or "highest"
    reader = SummaryReader(dirpath, pivot=False, extra_columns={'dir_name'})
    df = reader.scalars
    df = df.rename(columns={"tag": "metric", "dir_name": "fold"})
    grouped = df.groupby(["fold","metric"])
    # ['step'].max().reset_index() # this is choosing the max AUROC but we want the latest!!
    if select=='latest':
        df = df.loc[grouped['step'].idxmax()].reset_index(drop=True)
    elif select=='highest':
        df = df.loc[grouped['value'].idxmax()].reset_index(drop=True)
    elif select=='lowest':
        df = df.loc[grouped['value'].idxmin()].reset_index(drop=True)
    
    df = df[df["metric"].isin(metrics)].set_index(["fold"]).sort_values("metric").reset_index()

    res = df.pivot(index='fold', columns='metric', values='value')#.rename_axis(columns=None)
    if SAVE_PATH:
        with open(SAVE_PATH,'a') as f:
            f.write("\n\n")
            f.write(title)
            f.write("\n")
        res.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=True,floatfmt='.3f')
    return res

In [11]:

# Get pretraining task val metrics 
tasks=['fm','cl','pc','rc']
metric_names=['FM: AUPRC','CL: Accuracy','PC: MSE (deg)','RC: MSE (deg)']
metric_select=['highest','highest','lowest','lowest'] # which value to use when multiple checkpoints avail
varname = 'val_{task}_metric'
metrics=[varname.format(**locals()) for task in tasks]

pretrain_summary=[]
for i,metric in enumerate(metrics):
    name=metric_names[i]
    res=get_metric_results_from_eventfile(f"../lightning_logs/{MODEL_TYPE}_pretraining_seed{SEED}", [metric], SAVE_PATH, title=name, select="latest")
    pretrain_summary.append({"metric":name, 
                       "mean": res[[metric]].mean().item(),
                       "min": res[[metric]].min().item(),
                       "max": res[[metric]].max().item()
                       })
    


In [10]:
    
tit="Pretraining validation metrics: summarized"
with open(SAVE_PATH, "a") as f:
    f.write("\n\n")
    f.write(tit)
    f.write("\n")
tb = pd.DataFrame.from_records(pretrain_summary)
tb.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=False,floatfmt='.3f')

In [184]:
labels = ['Rote_X', 'Rote_Y', "Rote_Z", "Rote_D",
          'Inference_X', "Inference_Y","Inference_Z", "Inference_D", 
          "Deep_X", "Deep_Z",
          "MW"]

In [185]:
metrics = ["train_auroc", "val_auroc", "train_accuracy_epoch", "val_accuracy_epoch"]
val_aurocs = []
for label in labels:
    tit=f"# {label}"
    with open(SAVE_PATH, "a") as f:
        f.write("\n\n")
        f.write(tit)
        f.write("\n")
    res=get_metric_results_from_eventfile(f"../lightning_logs/{MODEL_TYPE}_{label}", metrics, SAVE_PATH, select="highest")
    res['label']=label
    val_aurocs.append({"label":label, 
                       "mean AUROC": res[['val_auroc']].mean().item(),
                       "min AUROC": res[['val_auroc']].min().item(),
                       "max AUROC": res[['val_auroc']].max().item()
                       })

In [186]:
tit="Held-out AUROCs"
with open(SAVE_PATH, "a") as f:
    f.write("\n\n")
    f.write(tit)
    f.write("\n")
tb = pd.DataFrame.from_records(val_aurocs)
tb.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=False,floatfmt='.3f')


In [199]:
# sequence lens
seqlens=[125, 250]
folds=range(4)
# Get pretraining task val metrics 
tasks=['fi','cl','pc','rc']
metric_names=['FI: AUROC','CL: Accuracy','PC: MSE (deg)','RC: MSE (deg)']
metric_select=['highest','highest','lowest','lowest'] # which value to use when multiple checkpoints avail
varname = 'val_{task}_metric'
metrics=[varname.format(**locals()) for task in tasks]

seqlen_summary=[]
for l in seqlens:
    THISLEN={"sequence length": l}
    for i,metric in enumerate(metrics):
        name=metric_names[i]
        per_fold=[]
        for f in folds:
            res=get_metric_results_from_eventfile(f"../lightning_logs/{MODEL_TYPE}_pre_seqlen_seed{SEED}/fold{f}seqlen{l}", [metric],  title=name, select="latest")
            per_fold.append(res[metric].item())
        THISLEN[name]=sum(per_fold)/len(per_fold)
    seqlen_summary.append(THISLEN)
        
# merge with pretraining_summary
pretrain_means={"sequence length": 500}
for entry in pretrain_summary:
    pretrain_means[entry['metric']]=entry['mean']

seqlen_summary.append(pretrain_means)

In [200]:
        
tit="GRU pretraining by sequence length"
with open(SAVE_PATH, "a") as f:
    f.write("\n\n")
    f.write(tit)
    f.write("\n")
tb = pd.DataFrame.from_records(seqlen_summary)
tb.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=False,floatfmt='.3f')

In [188]:
res[metric].item()

1.387425422668457